In [1]:
spark

In [2]:
infile = '/home/cloudera/2.kkbox_churn/raw_data/members_v3.csv'
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- city: string (nullable = true)
 |-- bd: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: string (nullable = true)
 |-- registration_init_time: string (nullable = true)



In [6]:
from pyspark.sql.functions import to_date, datediff, lit
df1 = df0.withColumn('days_since_init_regist',
               datediff(to_date(lit('2017-02-28')), to_date('registration_init_time', 'yyyyMMdd')))
df1.show(5)

+--------------------+----+---+------+--------------+----------------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|days_since_init_regist|
+--------------------+----+---+------+--------------+----------------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  null|            11|              20110911|                  1997|
|+tJonkh+O1CA796Fm...|   1|  0|  null|             7|              20110914|                  1994|
|cV358ssn7a0f7jZOw...|   1|  0|  null|            11|              20110915|                  1993|
|9bzDeJP6sQodK73K5...|   1|  0|  null|            11|              20110915|                  1993|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|                  1993|
+--------------------+----+---+------+--------------+----------------------+----------------------+
only showing top 5 rows



In [14]:
from pyspark.sql.functions import when, lit, col
df2 = df1.withColumn('age',
                      when((col('bd').cast('int')>=10) & (col('bd').cast('int')<=99), col('bd')) \
                      .otherwise(lit(None).cast('string'))
                    ) \
      .select('msno', 'city', 'age', 'gender', 'registered_via', 'days_since_init_regist')
df2.show(5)

+--------------------+----+----+------+--------------+----------------------+
|                msno|city| age|gender|registered_via|days_since_init_regist|
+--------------------+----+----+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|null|  null|            11|                  1997|
|+tJonkh+O1CA796Fm...|   1|null|  null|             7|                  1994|
|cV358ssn7a0f7jZOw...|   1|null|  null|            11|                  1993|
|9bzDeJP6sQodK73K5...|   1|null|  null|            11|                  1993|
|WFLY3s7z4EZsieHCt...|   6|  32|female|             9|                  1993|
+--------------------+----+----+------+--------------+----------------------+
only showing top 5 rows



In [15]:
outfile = '/home/cloudera/2.kkbox_churn/data01/feature_creation_01/members'
df2.write.format('csv').option('header', 'true').save(outfile)